In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import sys
sys.path.append("..")
import itertools
import time

import numpy as np

from santa.metrics import PerplexityCalculator
from santa.utils import save_text

/opt/mamba/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
scorer = PerplexityCalculator("google/gemma-2-9b")

Loading checkpoint shards: 100% 8/8 [00:08<00:00,  1.11s/it]


In [7]:
target_id = 4
if target_id == 0:
    text      = "reindeer mistletoe elf gingerbread family advent scrooge chimney fireplace ornament"
    sub_texts = []
    fix_ids   = []
elif target_id == 1:
    text      = "reindeer sleep walk the night and drive mistletoe scrooge laugh elf jump gingerbread bake ornament give family advent chimney fireplace"
    sub_texts = []
    fix_ids   = []
elif target_id == 2:
    text      = "sleigh yuletide beard carol cheer chimney decorations gifts grinch holiday holly jingle magi naughty nice nutcracker ornament polar workshop stocking"
    sub_texts = []
    fix_ids   = []
elif target_id == 3:
    text      = "sleigh of the magi yuletide cheer is unwrap gifts and eat cheer holiday decorations holly jingle relax sing carol visit workshop grinch naughty nice chimney stocking ornament nutcracker polar beard"
    sub_texts = []
    fix_ids   = [
        # "sleigh of the magi yuletide cheer",
        # "nutcracker polar beard",
        # "chimney stocking",
        # "grinch naughty",
        # "holly jingle",
        # "unwrap gifts"
        # "is unwrap gifts and eat cheer holiday decorations holly jingle relax sing carol visit workshop grinch naughty nice chimney stocking ornament nutcracker polar beard",
    ]
elif target_id == 4:
    text      = "of and to in the as you that it we with from have not night season eggnog milk chocolate candy peppermint cookie fruitcake toy doll game puzzle greeting card wrapping paper bow wreath poinsettia star angel snowglobe candle fireplace wish dream believe wonder hope joy peace merry hohoho kaggle workshop"
    sub_texts = [
        "night season eggnog milk chocolate candy peppermint cookie fruitcake toy doll game puzzle greeting card wrapping paper bow wreath poinsettia star angel snowglobe candle fireplace wish dream believe wonder hope joy peace merry hohoho kaggle workshop",
        # "candle fireplace wreath poinsettia angel star night wish dream", "from as peppermint candy fruitcake", "merry hohoho kaggle workshop",
        # "the of and to in that", "card wrapping paper bow", "chocolate milk eggnog", "toy doll game puzzle", "hope joy peace season", 
    ]
    fix_ids   = [14]
elif target_id == 5:
    text      = "from the the the of of and and not and to in is you that it with as advent card angel bake beard believe bow candy carol candle cheer cheer chocolate chimney chimney cookie decorations doll dream drive eat eggnog elf family fireplace fireplace fruitcake game gifts give gingerbread greeting grinch have holiday holly hohoho hope jingle jump joy kaggle laugh magi merry milk mistletoe naughty nice night night nutcracker ornament ornament paper peace peppermint polar poinsettia puzzle reindeer relax scrooge season sing sleigh sleep snowglobe star stocking toy unwrap visit walk we wish wonder workshop workshop wrapping wreath yuletide"
    sub_texts = [
        "you that it with as advent card angel bake beard believe bow candy carol candle cheer cheer chocolate chimney chimney cookie decorations doll dream drive eat eggnog elf family fireplace fireplace fruitcake game gifts give gingerbread greeting grinch have holiday holly hohoho hope jingle jump joy kaggle laugh magi merry milk mistletoe naughty nice night night nutcracker ornament ornament paper peace peppermint polar poinsettia puzzle reindeer relax scrooge season sing sleigh sleep snowglobe star stocking toy unwrap visit walk we wish wonder workshop workshop wrapping wreath yuletide",
        # "cookie decorations doll dream drive eat eggnog elf family fireplace fireplace fruitcake game gifts",
        # "snowglobe star stocking toy unwrap visit walk wish wonder workshop workshop wrapping wreath yuletide",
        # "kaggle laugh magi merry milk mistletoe naughty nice night night",
        # "nutcracker ornament ornament paper peace peppermint",
        # "puzzle reindeer relax scrooge season sing",
        # "angel bake beard believe bow",
        # "we the with you have advent card angel bake beard believe bow candy carol candle cheer cheer chocolate chimney chimney cookie decorations doll dream drive eat eggnog elf family fireplace fireplace fruitcake game gifts give gingerbread greeting grinch holiday holly hohoho hope jingle jump joy kaggle laugh magi merry milk mistletoe naughty nice night night not nutcracker ornament ornament paper peace peppermint polar poinsettia puzzle reindeer relax scrooge season sing sleigh sleep snowglobe star stocking toy unwrap visit walk wish wonder workshop workshop wrapping wreath yuletide",
    ]
    fix_ids   = [13]

In [8]:
scorer.get_perplexity(text)

67.47986016863359

In [9]:
for st in sub_texts:
    text = text.replace(st, "-".join(st.split()))
text, len(text.split())

('of and to in the as you that it we with from have not night-season-eggnog-milk-chocolate-candy-peppermint-cookie-fruitcake-toy-doll-game-puzzle-greeting-card-wrapping-paper-bow-wreath-poinsettia-star-angel-snowglobe-candle-fireplace-wish-dream-believe-wonder-hope-joy-peace-merry-hohoho-kaggle-workshop',
 15)

In [10]:
tokens = text.split()
np.array(tokens)[fix_ids]

array(['night-season-eggnog-milk-chocolate-candy-peppermint-cookie-fruitcake-toy-doll-game-puzzle-greeting-card-wrapping-paper-bow-wreath-poinsettia-star-angel-snowglobe-candle-fireplace-wish-dream-believe-wonder-hope-joy-peace-merry-hohoho-kaggle-workshop'],
      dtype='<U248')

In [11]:
len(tokens) - len(fix_ids)

14

In [12]:
def tokens2text(tokens, sep="-"):
    return " ".join(tokens).replace(sep, " ")


def beam_search(init_tokens, fix_ids, scorer, k=30, precomputed={}, n_iters=100):
    n = len(init_tokens)
    candidates = [init_tokens]
    best_score = np.inf
    best_text = None
    top_solutions = {}
    for iterations in range(n_iters):
        start = time.time()
        solutions = {}
        for candidate in candidates:
            for i in range(n):
                if i in fix_ids:
                    continue
                for j in range(i+1, n):
                    if j in fix_ids:
                        continue
                    s = list(candidate).copy()
                    s[i], s[j] = s[j], s[i]
                    s = tuple(s)
                    if s in precomputed:
                        continue
                    text = tokens2text(s)
                    score = scorer.get_perplexity(text)
                    precomputed[s] = score
                    solutions[s] = score
        solutions = sorted(solutions.items(), key=lambda x: x[1])[:k]
        top_solutions.update(solutions)
        if len(solutions) == 0:
            break
        candidates, scores = map(list, zip(*solutions))
        if scores[0] < best_score:
            best_score = scores[0]
            best_text = tokens2text(candidates[0])
        print(f"[{iterations} iterations] best: {best_score:.3f}, current: {scores[0]:.3f}, top10 mean: {np.mean(scores[:10]):.3f}, elapsed time: {time.time()-start:.1f}[s]")
        print(tokens2text(candidates[0]))
    return best_score, best_text, top_solutions

In [ ]:
%%time
best_score, best_text, top_solutions = beam_search(tokens, fix_ids, scorer, k=20, n_iters=20)

[0 iterations] best: 70.821, current: 70.821, top10 mean: 73.801, elapsed time: 6.9[s]
of and to in the as you that it we with from not have night season eggnog milk chocolate candy peppermint cookie fruitcake toy doll game puzzle greeting card wrapping paper bow wreath poinsettia star angel snowglobe candle fireplace wish dream believe wonder hope joy peace merry hohoho kaggle workshop
[1 iterations] best: 67.480, current: 67.480, top10 mean: 73.602, elapsed time: 117.0[s]
of and to in the as you that it we with from have not night season eggnog milk chocolate candy peppermint cookie fruitcake toy doll game puzzle greeting card wrapping paper bow wreath poinsettia star angel snowglobe candle fireplace wish dream believe wonder hope joy peace merry hohoho kaggle workshop
[2 iterations] best: 67.480, current: 75.937, top10 mean: 76.852, elapsed time: 120.9[s]
of and to the as in you that it we with from not have night season eggnog milk chocolate candy peppermint cookie fruitcake toy do

In [ ]:
scorer.get_perplexity(best_text)

In [ ]:
best_score

In [ ]:
best_text

In [ ]:
tmp = sorted(top_solutions.items(), key=lambda x: x[1])
candidates, scores = map(list, zip(*tmp))
candidates = [tokens2text(candidate) for candidate in candidates]

In [ ]:
candidates[0]

In [ ]:
candidates[1]

In [ ]:
import matplotlib.pyplot as plt

plt.plot(np.arange(len(scores)), scores)

In [33]:
import math
from pathlib import Path

output_dir = "./output"
output_dir = Path(output_dir, f"id{target_id}")
output_dir.mkdir(parents=True, exist_ok=True)
for no, (text, score) in enumerate(zip(candidates, scores)):
    f, i = math.modf(score)
    i = int(i)
    f = int(f * 100000)
    with open(Path(output_dir, f"id{target_id}_{no:0>4}_{i:0>4}.{f}.txt"), "w") as f:
        f.write(text)

In [34]:
output_dir = "./output"
save_text(best_text, best_score, target_id, output_dir=output_dir)

sleigh yuletide beard carol cheer chimney decorations gifts grinch holiday holly jingle magi naughty nice nutcracker ornament polar workshop stocking
